In [2]:
import torch
import torch.nn as nn

softmax = nn.Softmax(-1)
w_gate = torch.randn(4, 6)
input = torch.randn(2, 4)
logits = torch.mm(input,w_gate)
top_k_logits, top_k_indices = logits.topk(2, dim=-1)
top_k_gates = softmax(top_k_logits)
zeros = torch.zeros_like(logits, requires_grad=True)
gates = zeros.scatter(-1, top_k_indices, top_k_gates)
print("logits=", logits)
print("gates=", gates)

logits= tensor([[ 1.2533,  0.4426,  1.5564, -1.6071, -0.6329, -0.8361],
        [ 0.5491, -0.5321,  0.4401, -1.5789,  3.9186, -1.5876]])
gates= tensor([[0.4248, 0.0000, 0.5752, 0.0000, 0.0000, 0.0000],
        [0.0333, 0.0000, 0.0000, 0.0000, 0.9667, 0.0000]],
       grad_fn=<ScatterBackward0>)


In [15]:
all_expert = torch.nonzero(gates)
print("all_expert=", all_expert)
sorted_experts, index_sorted_experts = all_expert.sort(0)
print("sorted_experts=", sorted_experts)
print("index_sorted_experts=", index_sorted_experts)
_, _expert_index = sorted_experts.split(1, dim=1)
print("_expert_index=", _expert_index)
batch_index = all_expert[index_sorted_experts[:, 1],0]
print("batch_index=", batch_index)
part_sizes = (gates > 0).sum(0).tolist()
print("part_size=", part_sizes)
gates_exp = gates[batch_index]
print("gates_exp=", gates_exp)
_nonzero_gates = torch.gather(gates_exp, 1, _expert_index)
print("_nonzero_gates", _nonzero_gates)


all_expert= tensor([[0, 0],
        [0, 2],
        [1, 0],
        [1, 4]])
sorted_experts= tensor([[0, 0],
        [0, 0],
        [1, 2],
        [1, 4]])
index_sorted_experts= tensor([[0, 0],
        [1, 2],
        [2, 1],
        [3, 3]])
_expert_index= tensor([[0],
        [0],
        [2],
        [4]])
batch_index= tensor([0, 1, 0, 1])
part_size= [2, 0, 1, 0, 1, 0]
gates_exp= tensor([[0.4248, 0.0000, 0.5752, 0.0000, 0.0000, 0.0000],
        [0.0333, 0.0000, 0.0000, 0.0000, 0.9667, 0.0000],
        [0.4248, 0.0000, 0.5752, 0.0000, 0.0000, 0.0000],
        [0.0333, 0.0000, 0.0000, 0.0000, 0.9667, 0.0000]],
       grad_fn=<IndexBackward0>)
_nonzero_gates tensor([[0.4248],
        [0.0333],
        [0.5752],
        [0.9667]], grad_fn=<GatherBackward0>)


In [19]:
experts_from, experts_used = all_expert.split(1,dim=1)
print(experts_from)
experts_used, experts_used_index = experts_used.sort(0)
experts_from = experts_from[experts_used_index.flatten()]
print(experts_from)


tensor([[0],
        [0],
        [1],
        [1]])
tensor([[[0]],

        [[1]],

        [[0]],

        [[1]]])


In [ ]:
experts_used = experts_index[-1]  # [num_used_experts]
experts_from = torch.stack(experts_index[:-1])  # [len(gates.shape), num_used_experts]
experts_used, index_sorted_experts = experts_used.sort(stable=True)  # [num of used experts]
experts_from = list(experts_from[..., index_sorted_experts])  # [tensor(num_used_experts), tensor(num_used_experts)]
experts_gate = experts_gate[index_sorted_experts]  # [nus]
experts_count = list(gates.reshape(-1, self.num_experts).count_nonzero(0))
  # [num_epxerts]
experts_input = x[experts_from]  # [nus, input_size]
experts_input = torch.split(experts_input, experts_count, 0)
experts_output = [self.experts[i](experts_input[i]) for i in range(self.num_experts)]
experts_output = torch.cat(experts_output)  # [nus, output_size]
experts_output *= experts_gate.unsqueeze(-1)  # [nus, output_size]
# zeros = torch.zeros(x.shape[0], x.shape[1], self.output_size).cpu()  # [batch_size, ..., output_size]
# zeros[experts_from] += experts_output.cpu()
zeros = torch.zeros(x.shape[0], x.shape[1], self.output_size)  # [batch_size, ..., output_size]
zeros[experts_from] += experts_output
zeros = zeros.to(self.device)
zeros = self.dropout(self.sigmod(zeros)) + x